# Input Functions

In [ ]:
!pip install tensorflow_addons
from google.colab import drive
drive.mount("/content/gdrive")
!unzip gdrive/My\ Drive/archive.zip > /dev/null
!unzip gdrive/My\ Drive/saved_models.zip > /dev/null

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 15.5 MB/s eta 0:00:00
Mounted at /content/gdrive


In [ ]:
! git clone -b mask https://github.com/WongKinYiu/yolov7.git
! pip install pyyaml==5.1
! pip install 'git+https://github.com/facebookresearch/detectron2.git'
! pip install torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html

Cloning into 'yolov7'...
remote: Enumerating objects: 1148, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 1148 (delta 4), reused 0 (delta 0), pack-reused 1139
Receiving objects: 100% (1148/1148), 70.42 MiB | 27.50 MiB/s, done.
Resolving deltas: 100% (490/490), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp310-cp310-linux_x86_64.whl size=44090 sha256=739969e034ccfa1fee870f7a8eddd2bfaaa701c72e7ac474ac541d2592990e0d
  Stored in directory: /root/.cache/pip/wheels/70/83/31/975b737609aba39a4099d471d5684141c1fdc3404f97e7f68a
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: p

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
from keras import backend
from keras import datasets, layers, models, initializers, activations
from keras.layers import Conv2D, BatchNormalization, Activation, Conv2DTranspose, Concatenate
from functools import reduce
import torch
from torchvision import transforms
from PIL import Image
from matplotlib import cm

from skimage import measure
from skimage import transform
from skimage.io import imread, imsave, imshow
from skimage.transform import resize
from skimage.filters import gaussian
from skimage.morphology import dilation, disk
from skimage.draw import polygon, polygon_perimeter

print(f'Tensorflow version {tf.__version__}')
print(f'GPU is {"ON" if tf.compat.v1.config.experimental.list_physical_devices("GPU") else "OFF" }')


import numpy as np
%matplotlib inline
import matplotlib, random
import matplotlib.pyplot as plt
import pandas as pd 
import cv2
import os

from imutils import paths
import pathlib
import time
import glob
from natsort import natsorted
import tarfile
import tqdm

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Tensorflow version 2.12.0
GPU is ON


In [ ]:
add_signs_classes = {'0':183,
                     '1':184,
                     '2':185,
                     '3':186,
                     '4':187,
                     '5':188,
                     '6':189,
                     '7':190,
                     '8':191,
                     '9':192,
                     '10':193,
                     '11':194,
                     '12':195,
                     '13':196,
                     '14':197,
                     '15':198,
                     '16':199,
                     '17':200,
                     '18':201,
                     '19':202,
                     '20':203,
                     '21':204}

In [ ]:
forb_signs_classes = {'0':0,
'1':1,
'2':10,
'3':15,
'4':16,
'5':17,
'6':2,
'7':3,
'8':32,
'9':4,
'10':41,
'11':42,
'12':5,
'13':6,
'14':7,
'15':72,
'16':73,
'17':74,
'18':75,
'19':76,
'20':77,
'21':78,
'22':79,
'23':8,
'24':80,
'25':81,
'26':82,
'27':83,
'28':84,
'29':85,
'30':86,
'31':87,
'32':88,
'33':89,
'34':9,
'35':90,
'36':91,
'37':92,
'38':93,
'39':94,
'40':95,
'41':96,
'42':97,}

In [ ]:
Warning_signs_classes = {'0':18,
'1':19,
'2':20,
'3':21,
'4':22,
'5':23,
'6':24,
'7':25,
'8':26,
'9':27,
'10':28,
'11':29,
'12':30,
'13':31,
'14':48,
'15':49,
'16':50,
'17':51,
'18':52,
'19':53,
'20':54,
'21':55,
'22':56,
'23':57,
'24':58,
'25':59,
'26':60,
'27':61,
'28':62,
'29':63,
'30':64,
'31':65,
'32':66,
'33':67,
'34':68,
'35':69,
'36':70,
'37':71,}

In [ ]:
inf_signs_classes = {'0':148,
'1':149,
'2':150,
'3':151,
'4':152,
'5':153,
'6':154,
'7':155,
'8':156,
'9':157,
'10':158,
'11':159,
'12':160,
'13':161,
'14':162,
'15':163,
'16':164,}

In [ ]:
presc_signs_classes = {'0':100,
'1':101,
'2':102,
'3':103,
'4':104,
'5':105,
'6':106,
'7':107,
'8':108,
'9':109,
'10':33,
'11':34,
'12':35,
'13':36,
'14':37,
'15':38,
'16':39,
'17':40,
'18':98,
'19':99,}

In [ ]:
priority_signs_classes = {'0':12,
'1':13,
'2':14,
'3':43,
'4':44,
'5':45,
'6':46,
'7':47,}

In [ ]:
service_signs_classes = {'0':165,
'1':166,
'2':167,
'3':168,
'4':169,
'5':170,
'6':171,
'7':172,
'8':173,
'9':174,
'10':175,
'11':176,
'12':177,
'13':178,
'14':179,
'15':180,
'16':181,
'17':182,}

In [ ]:
special_instructions_signs_classes = {'0':110,
'1':111,
'2':112,
'3':113,
'4':114,
'5':115,
'6':116,
'7':117,
'8':118,
'9':119,
'10':120,
'11':121,
'12':122,
'13':123,
'14':124,
'15':125,
'16':126,
'17':127,
'18':128,
'19':129,
'20':130,
'21':131,
'22':132,
'23':133,
'24':134,
'25':135,
'26':136,
'27':137,
'28':138,
'29':139,
'30':140,
'31':141,
'32':142,
'33':143,
'34':144,
'35':145,
'36':146,
'37':147,}

In [ ]:
def run_inference(url, model):
    image = cv2.imread(url)
    image = cv2.resize(image,(640,384))
    image = letterbox(image, 640, stride=64, auto=True)[0] 
    image = transforms.ToTensor()(image)
    image = image.half().to(device)
    image = image.unsqueeze(0)
    output = model(image)
    return output, image
  
def re_parammetatizing(inf_out, attn, bases, sem_output, image, model):
  bases = torch.cat([bases, sem_output], dim=1)
  nb,_, height, width = image.shape
  names = model.names
  pooler_scale = model.pooler_scale

  pooler = ROIPooler(output_size=hyp['mask_resolution'], 
                    scales=(pooler_scale,), 
                    sampling_ratio=1, 
                    pooler_type='ROIAlignV2', 
                    canonical_level=2)
                    
  # output, output_mask, output_mask_score, output_ac, output_ab
  output, output_mask, _, _, _ = non_max_suppression_mask_conf(inf_out, attn, bases, pooler, hyp,
                                                            conf_thres=0.25, iou_thres=0.65, merge=False, mask_iou=None) 
  return  output, output_mask, height, width, names

def load_model(weights_path):
    model = torch.load(weights_path, map_location=device)['model']
    model.eval()

    if torch.cuda.is_available():
        model.half().to(device)
    return model

def crop_image(x1,y1,x2,y2, first_image):
  im = Image.fromarray(first_image)
  img2 = im.crop((x1,y1,x2,y2))
  img = np.array(img2)
  return img

In [ ]:
def get_colors(mask):
    my_array = mask
    my_array[my_array == 10 ] = 0
    my_array[my_array == 18 ] = 0
    my_array[my_array == 12 ] = 0
    return my_array

In [ ]:
img_size = 640
semantic_image_size = (img_size, 384)
output_size = (img_size, 384)

num_classes = 23
colors = np.array([(0, 0, 0),
                   (70, 70, 70),
                   (100, 40, 40),
                   (55, 90, 80),
                   (220, 20, 60),
                   (153, 153, 153),
                   (157, 234, 50),
                   (128, 64, 128),
                   (244, 35, 232),
                   (107, 142, 35),
                   (0, 0, 142),
                   (102, 102, 156),
                   (220, 220, 0),
                   (70, 130, 180),
                   (81, 0, 81),
                   (150, 100, 100),
                   (230, 150, 140),
                   (180, 165, 180),
                   (250, 170, 30), 
                   (110, 190, 160),
                   (170, 120, 50),
                   (45, 60, 150),
                   (145, 170, 100)])

# Load Panoptic Model

## Load Semantic head (EfficientUDet+)

In [ ]:
from EfficientUDet import Semantic_model
from keras.optimizers import SGD, Adam
from keras import metrics


Tensorflow version 2.12.0
GPU is ON


In [ ]:
opt = Adam(learning_rate=0.0003)
Semantic_model = Semantic_model(0, num_classes=num_classes)
Semantic_model.compile(optimizer=opt,loss='binary_crossentropy', metrics=[metrics.OneHotIoU(num_classes,[i for i in range(0,num_classes)])])


In [ ]:
Semantic_model.load_weights('./gdrive/MyDrive/sem_weights(0.6044).h5')


## Load Instance Head (YoloV7)

In [ ]:
%cd yolov7
! curl -L https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-mask.pt -o yolov7-mask.pt

/content/yolov7
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 87.0M  100 87.0M    0     0  46.0M      0  0:00:01  0:00:01 --:--:-- 69.9M


In [ ]:
from utils.datasets import letterbox
from utils.general import non_max_suppression_mask_conf
 
from detectron2.modeling.poolers import ROIPooler
from detectron2.structures import Boxes
from detectron2.utils.memory import retry_if_cuda_oom
from detectron2.layers import paste_masks_in_image

In [ ]:
import yaml
with open('../yolov7/data/hyp.scratch.mask.yaml') as f:
    hyp = yaml.load(f, Loader=yaml.FullLoader)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Instance_model = load_model('yolov7-mask.pt')

## Load Classification Heads (ResNets)

### Traffic Signs Model

In [ ]:
from keras import models
import tensorflow_addons as tfa

# categorical model
Trafic_Sign_model = models.load_model('../saved_models/Trafic_Signs_CNN_categorical.h5')

# Signs model
add_signs_model = models.load_model('../saved_models/Trafic_Signs_CNN_add_signs.h5')
forb_signs_model = models.load_model('../saved_models/Trafic_Signs_CNN_forb_signs.h5')
inf_signs_model = models.load_model('../saved_models/Trafic_Signs_CNN_inf_signs.h5')
presc_signs_model = models.load_model('../saved_models/Trafic_Signs_CNN_presc_signs.h5')
priority_signs_model = models.load_model('../saved_models/Trafic_Signs_CNN_priority_signs.h5')
service_signs_model = models.load_model('../saved_models/Trafic_Signs_CNN_service_signs.h5')
special_instructions_signs_model = models.load_model('../saved_models/Trafic_Signs_CNN_special_instructions_signs.h5')
Warning_signs_model = models.load_model('../saved_models/Trafic_Signs_CNN_Warning_signs.h5')

### Traffic Lights Model

In [ ]:
Trafic_Lights_model = models.load_model('../saved_models/Trafic_Lights_CNN.h5')


### Road Type Model

In [ ]:


# categorical model
Road_Type_model = models.load_model('../saved_models/Road_Type_AllClasses_CNN.h5')

# road model
Road_Type_Asphalt_model = models.load_model('../saved_models/Road_Type_Asphalt_CNN.h5')
Road_Type_Concrete_model = models.load_model('../saved_models/Road_Type_Concrete_CNN.h5')
Road_Type_Gravel_model = models.load_model('../saved_models/Road_Type_Gravel_CNN.h5')
Road_Type_Mud_model = models.load_model('../saved_models/Road_Type_Mud_CNN.h5')
Road_Type_Snow_model = models.load_model('../saved_models/Road_Type_Snow_CNN.h5')

### Road Line Model

In [ ]:
Road_Line_model = models.load_model('../saved_models/Road_Line_CNN.h5')

# Model Predicition

In [ ]:
def instance_model_prediction(image_path, model):
    
    output, image = run_inference(image_path, model)
    inf_out = output['test']
    attn = output['attn']
    bases = output['bases']
    sem_output = output['sem']

    output, output_mask, height, width, names = re_parammetatizing(inf_out, attn, bases, sem_output, image, model)
    pred, pred_masks = output[0], output_mask[0]
    base = bases[0]
    bboxes = Boxes(pred[:, :4])

    original_pred_masks = pred_masks.view(-1, hyp['mask_resolution'], hyp['mask_resolution'])
    original_image = np.moveaxis(image.cpu().numpy().squeeze(), 0, 2).astype('float32')
    original_image = cv2.cvtColor(original_image, cv2.COLOR_RGB2BGR)

    pred_masks = retry_if_cuda_oom(paste_masks_in_image)(original_pred_masks, bboxes, (height, width), threshold=0.5)

    pred_masks_np = pred_masks.detach().cpu().numpy()
    pred_cls = pred[:, 5].detach().cpu().numpy()
    pred_conf = pred[:, 4].detach().cpu().numpy()
    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)
    nbboxes = bboxes.tensor.detach().cpu().numpy().astype(np.int)
    return pred_masks_np, pred_cls, pred_conf, nimg, nbboxes, height, width, original_image, names

def semantic_model_prediction(img, output_size):
    image = cv2.imread(img)
    img = cv2.resize(image, output_size)
    pred = Semantic_model(np.array([img]))
    pred = np.argmax (pred, axis=-1) [0, :,:]
    return pred

In [ ]:
def panoptic_fusion(pred, original_image, pred_masks_np, height, width, pred_conf):

    panoptic_pred = get_colors(pred)
    for i in range(len(instance_pred_masks_np)):
      if pred_conf[i]>=0.5:
        if instance_pred_cls[i] == 2 or instance_pred_cls[i] == 7:
          panoptic_pred[instance_pred_masks_np[i]] = panoptic_pred[instance_pred_masks_np[i]]  + np.array(10, dtype=np.uint8)
        if instance_pred_cls[i] == 9:
          panoptic_pred[instance_pred_masks_np[i]] = panoptic_pred[instance_pred_masks_np[i]]  + np.array(12, dtype=np.uint8)
    
    return panoptic_pred

def panoptic_bounding(panoptic_image, original_image, pred_masks_np, names, pred_cls, pred_conf, image_path, nbboxes):
    height,width = panoptic_image.shape[:2]
    img = cv2.imread(image_path)
    original_image = cv2.resize(img, (width,height))
    orig_img = original_image.copy()
    pred_img = cv2.addWeighted(colors[panoptic_image].astype(np.int32), 0.8,original_image.astype(np.int32), 1.,0)
    for i in range(len(pred_masks_np)):
        pred_img = pred_img.copy()
        if pred_conf[i]>=0.5:
          color = [np.random.randint(255), np.random.randint(255), np.random.randint(255)]
          pred_img = cv2.rectangle(pred_img, (nbboxes[i][0], nbboxes[i][1]), (nbboxes[i][2], nbboxes[i][3]), color, 2)
          if int(pred_cls[i]) == 9:
            traffic_light_image = crop_image(nbboxes[i][0], nbboxes[i][1], nbboxes[i][2], nbboxes[i][3], original_image)
            %matplotlib inline
            traffic_light_image = cv2.cvtColor(traffic_light_image, cv2.COLOR_BGR2RGB)
            # plt.imshow(traffic_light_image)
            # plt.savefig('../%s.png' % (random.randint(0,100)),bbox_inches='tight')
            pred_label = traffic_sign_prediction(traffic_light_image)
            label = '%s %.3f' % (pred_label, pred_conf[i])
          else: 
            label = '%s %.3f' % (names[int(pred_cls[i])], pred_conf[i])
          t_size = cv2.getTextSize(label, 0, fontScale=0.1, thickness=1)[0]
          c2 = nbboxes[i][0] + t_size[0], nbboxes[i][1] - t_size[1] - 3
          pred_img = cv2.rectangle(pred_img, (nbboxes[i][0], nbboxes[i][1]), c2, color, -1, cv2.LINE_AA)
          pred_img = cv2.putText(pred_img, label, (nbboxes[i][0], nbboxes[i][1] - 2), 0, 0.5, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)
    label_line = str(road_line_prediction(orig_img))
    color = [np.random.randint(255), np.random.randint(255), np.random.randint(255)]
    pred_img = cv2.rectangle(pred_img, (0, 250), (250, 320), color, 2, cv2.LINE_AA)
    pred_img = cv2.putText(pred_img, label_line, (0, 250 - 2), 0, 0.5, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)

    label_road = str(road_type_prediction(orig_img))
    color = [np.random.randint(255), np.random.randint(255), np.random.randint(255)]
    pred_img = cv2.rectangle(pred_img, (0, 250), (550, 320), color, 2, cv2.LINE_AA)
    pred_img = cv2.putText(pred_img, label_road, (400, 250 - 2), 0, 0.5, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)
    panoptic_bboxes_image = np.array(pred_img)
    
    return panoptic_bboxes_image


In [ ]:
def traffic_sign_prediction(image):
  traffic_light_image = cv2.resize (image, (30,60))
  traffic_light_image = cv2.cvtColor(traffic_light_image, cv2.COLOR_BGR2RGB)
  traffic_light_image = np.array(traffic_light_image)
  img_batch = np.expand_dims(traffic_light_image,0)
  prediction = Trafic_Lights_model.predict(img_batch)
  prediction = np.argmax(prediction, axis=1)
  if prediction[0] == 3:p = 'yellow_traffic_light_signal'
  if prediction[0] == 1:p = 'red_traffic_light_signal'
  if prediction[0] == 2:p = 'green_traffic_light_signal' 
  if prediction[0] == 0:p = 'back_traffic_light'
  return p

def road_line_prediction(image):
  road_image = image[300:450, 0:250]
  road_image = cv2.cvtColor(road_image,cv2.COLOR_BGR2RGB) ## переводим в оттенки серого
  road_image = cv2.resize(road_image, (256,256))

  plt.imshow(road_image)
  road_image = np.array(road_image)
  img_batch = np.expand_dims(road_image,0)
  prediction = np.argmax(Road_Line_model.predict(img_batch), axis=1)
  if prediction[0] == 0:p = 'Botts-dots '
  if prediction[0] == 3:p = 'continuous'
  if prediction[0] == 2:p = 'continuous_yellow'
  if prediction[0] == 1:p = 'dashed'
  if prediction[0] == 4:p = 'double_continuous'
  if prediction[0] == 5:p = 'double_dashed'

  return p

def traffic_signs_prediction(image):
  image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB) ## переводим в оттенки серого
  image_resized = transform.resize(image, (60,60))
  img_batch = np.expand_dims(image_resized,0)

  # Начинаем предсказание категории знака
  categorical_prediction = Trafic_Sign_model.predict(img_batch)
  categorical_prediction = np.argmax(categorical_prediction, axis=1)
  # Начинаем предсказание знака внутри категории
  if categorical_prediction[0] == 2:
      image_resized = transform.resize(image, (60,90))
      img_batch = np.expand_dims(image_resized,0)
      
      add_signs_prediction = add_signs_model.predict(img_batch)
      add_signs_prediction = np.argmax(add_signs_prediction, axis=1)
      
      return add_signs_classes[str(add_signs_prediction[0])]
      
  if categorical_prediction[0] == 0:
      forb_signs_prediction = forb_signs_model.predict(img_batch)
      forb_signs_prediction = np.argmax(forb_signs_prediction, axis=1)
      
      return  forb_signs_classes[(str(forb_signs_prediction[0]))]

  if categorical_prediction[0] == 3:
      inf_signs_prediction = inf_signs_model.predict(img_batch)
      inf_signs_prediction = np.argmax(inf_signs_prediction, axis=1)
      
      return  inf_signs_classes[(str(inf_signs_prediction[0]))]
            
  if categorical_prediction[0] == 4:
      presc_signs_prediction = presc_signs_model.predict(img_batch)
      presc_signs_prediction = np.argmax(presc_signs_prediction, axis=1)
      return  presc_signs_classes[(str(presc_signs_prediction[0]))]
      
  if categorical_prediction[0] == 5:
      image_resized = transform.resize(image, (70,70))
      img_batch = np.expand_dims(image_resized,0)
      
      priority_signs_prediction = priority_signs_model.predict(img_batch)
      priority_signs_prediction = np.argmax(priority_signs_prediction, axis=1)
      
      return  priority_signs_classes[(str(priority_signs_prediction[0]))]

  if categorical_prediction[0] == 6:
      image_resized = transform.resize(image, (120,80))
      img_batch = np.expand_dims(image_resized,0)
      
      service_sign_prediction = service_signs_model.predict(img_batch)
      service_sign_prediction = np.argmax(service_sign_prediction, axis=1)
      
      return service_signs_classes[(str(service_sign_prediction[0]))]
         
  if categorical_prediction[0] == 7:

      special_instructions_signs_prediction = special_instructions_signs_model.predict(img_batch)
      special_instructions_signs_prediction = np.argmax(special_instructions_signs_prediction, axis=1)
      
      return  special_instructions_signs_classes[(str(special_instructions_signs_prediction[0]))]

          
  if categorical_prediction[0] == 1:
      Warning_signs_prediction = Warning_signs_model.predict(img_batch)
      Warning_signs_prediction = np.argmax(Warning_signs_prediction, axis=1)
      
      return Warning_signs_classes[(str(Warning_signs_prediction[0]))]

def road_type_prediction(image):
  road_image = cv2.resize(image[300:450, 0:550], (240,360))
  road_image = cv2.cvtColor(road_image,cv2.COLOR_BGR2RGB) ## переводим в оттенки серого
  plt.imshow(road_image)
  img_batch = np.expand_dims(road_image,0)
  road_class = np.argmax(Road_Type_model.predict(img_batch), axis=1)
 
  # Начинаем предсказание знака внутри категории
  if road_class[0] == 0:
      Road_Type_Snow_prediction = Road_Type_Snow_model.predict(img_batch)
      Road_Type_Snow_prediction = np.argmax(Road_Type_Snow_prediction, axis=1)
      
      if Road_Type_Snow_prediction[0] ==0: p = 'fresh_snow'
      if Road_Type_Snow_prediction[0] ==1: p = 'ice'
      if Road_Type_Snow_prediction[0] ==2: p = 'melted_snow'

      return p
  if road_class[0] == 1:
      Road_Type_Mud_prediction = Road_Type_Mud_model.predict(img_batch)
      Road_Type_Mud_prediction = np.argmax(Road_Type_Mud_prediction, axis=1)

      if Road_Type_Mud_prediction[0] ==0: p = 'dry_mud'
      if Road_Type_Mud_prediction[0] ==1: p = 'water_mud'
      if Road_Type_Mud_prediction[0] ==2: p = 'wet_mud'
      
      return p
  
  if road_class[0] == 2:
    Road_Type_Gravel_prediction = Road_Type_Gravel_model.predict(img_batch)
    Road_Type_Gravel_prediction = np.argmax(Road_Type_Gravel_prediction, axis=1)

    if Road_Type_Gravel_prediction[0] ==0: p = 'dry_gravel'
    if Road_Type_Gravel_prediction[0] ==1: p = 'water_gravel'
    if Road_Type_Gravel_prediction[0] ==2: p = 'wet_gravel'
    
    return p
  if road_class[0] == 3:
    Road_Type_Concrete_prediction = Road_Type_Concrete_model.predict(img_batch)
    Road_Type_Concrete_prediction = np.argmax(Road_Type_Concrete_prediction, axis=1)
    
    if Road_Type_Concrete_prediction[0] ==0: p = 'dry_concrete'
    if Road_Type_Concrete_prediction[0] ==1: p = 'water_concrete'
    if Road_Type_Concrete_prediction[0] ==2: p = 'wet_concrete'
    
    return p
      
  if road_class[0] == 4:
      Road_Type_Asphalt_prediction = Road_Type_Asphalt_model.predict(img_batch)
      Road_Type_Asphalt_prediction = np.argmax(Road_Type_Asphalt_prediction, axis=1)
      if Road_Type_Asphalt_prediction[0] ==0: p = 'dry_asphalt'
      if Road_Type_Asphalt_prediction[0] ==1: p = 'water_asphalt'
      if Road_Type_Asphalt_prediction[0] ==2: p = 'wet_asphalt'
      return  p

In [ ]:
!pip install natsort

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from natsort import natsorted


In [ ]:
images_path = natsorted(glob.glob('../dataA/dataA/CameraRGB/*.png'))
masks_path = natsorted(glob.glob('../dataA/dataA/CameraSeg/*.png'))
count = 0
all_pred = []
for i in range(138,151):
  try:
    image_path = str(images_path[i]) 
    semantic_pred = semantic_model_prediction (image_path, semantic_image_size)
    semantic_pred_2 = semantic_model_prediction (image_path, semantic_image_size)

    true_mask = cv2.imread(masks_path[i])
    true_mask = cv2.resize(true_mask, semantic_image_size)
    true_mask = true_mask[:,:,2]
    
    instance_pred_masks_np, instance_pred_cls, instance_pred_conf, nimg, nbboxes, height, width,original_image, names = instance_model_prediction(image_path, Instance_model)
    panoptic_image = panoptic_fusion(semantic_pred, original_image, instance_pred_masks_np, height, width,instance_pred_conf)

    iou = tf.keras.metrics.IoU(num_classes=23, target_class_ids=[0,1,7,8,9,10,11,12,15,16,18,19,20,21,22])
    iou.update_state(panoptic_image[:310], true_mask[:310])
    print(iou.result().numpy())
    all_pred.append(iou.result().numpy())

    %matplotlib inline

    plt.figure(figsize = (20,20))
    plt.subplot (1,2,1)
    plt.title("Original Image")
    plt.axis('off')
    plt.imshow (original_image)

    # plt.subplot (1,5,2)
    # plt.title("Ground Truth")
    # plt.axis('off')
    # plt.imshow (colors [true_mask])

    # plt.subplot (1,5,3)
    # plt.title("Semantic Prediction")
    # plt.axis('off')
    # plt.imshow (colors [semantic_pred_2])

    # plt.subplot (1,5,4)
    # plt.title("Panoptic Prediction")
    # plt.axis('off')
    # plt.imshow (colors[panoptic_image])

    plt.subplot (1,2,2)
    plt.title("Hydra Panoptic Prediction")
    plt.axis('off')
    plt.imshow (panoptic_bounding(panoptic_image, original_image, instance_pred_masks_np, names, instance_pred_cls, instance_pred_conf, image_path, nbboxes))

    plt.savefig('../gdrive/MyDrive/z/panoptic %s.png' % i,bbox_inches='tight')
    plt.show()
  except:
    pass
print(sum(all_pred)/len(all_pred))


Output hidden; open in https://colab.research.google.com to view.